In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from random import shuffle
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten 
from keras.layers import Dense, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import split_folders

In [2]:
split_folders.ratio('mask_data', output = 'final_data', ratio = (0.8, 0.2))

Copying files: 1376 files [00:01, 1359.39 files/s]


In [3]:
classifier = Sequential()

In [4]:
classifier.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [5]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 21632)             0

In [6]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [8]:
TRAINING_DIR = 'final_data/train'
VALIDATION_DIR = 'final_data/val'

In [9]:
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=16,
    class_mode='binary',
    target_size=(224, 224))

Found 1100 images belonging to 2 classes.


In [10]:
validation_datagen = ImageDataGenerator(
                        rescale=1. / 255)

In [11]:
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size=16,
    class_mode='binary',
    target_size=(224, 224))


Found 276 images belonging to 2 classes.


In [12]:
classifier.fit(train_generator,
            epochs=15,
            steps_per_epoch=1100/16,
            verbose=1,
            validation_data=validation_generator,
            validation_steps=276/16)

Epoch 1/15
69/68 [==============================] - 35s 509ms/step - loss: 0.5181 - accuracy: 0.7545 - val_loss: 0.2282 - val_accuracy: 0.9058
Epoch 2/15
69/68 [==============================] - 29s 426ms/step - loss: 0.2805 - accuracy: 0.8991 - val_loss: 0.1442 - val_accuracy: 0.9348
Epoch 3/15
69/68 [==============================] - 29s 426ms/step - loss: 0.2341 - accuracy: 0.9127 - val_loss: 0.1149 - val_accuracy: 0.9565
Epoch 4/15
69/68 [==============================] - 29s 427ms/step - loss: 0.1619 - accuracy: 0.9482 - val_loss: 0.1515 - val_accuracy: 0.9420
Epoch 5/15
69/68 [==============================] - 29s 426ms/step - loss: 0.1599 - accuracy: 0.9427 - val_loss: 0.0519 - val_accuracy: 0.9891
Epoch 6/15
69/68 [==============================] - 29s 427ms/step - loss: 0.1323 - accuracy: 0.9500 - val_loss: 0.1029 - val_accuracy: 0.9565
Epoch 7/15
69/68 [==============================] - 30s 428ms/step - loss: 0.0980 - accuracy: 0.9700 - val_loss: 0.0274 - val_accuracy: 0.9891

In [20]:
classifier.save("mask_detector.h5", save_format="h5")

In [25]:
for test_image in os.listdir('testing/'):
    test_image = image.load_img('testing/'+test_image, target_size = (224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    print(result[0][0])
    if result[0][0] == 1:
        prediction = "Not Masked!"
        print(prediction)
    else: 
        prediction = "Masked"
        print(prediction)
    

0.0
Masked
0.0
Masked
0.0
Masked
1.0
Not Masked!
1.0
Not Masked!
1.0
Not Masked!
